In [1]:
import os

In [2]:
class gate_dlinkedList:
    def __init__(self, gate_id):
        self.gate_id = gate_id
        self.lock = False
        self.next = None
        self.prev = None
        self.gain = None
    
    def calculate_gain(self, current_gate):
        prev_count= 0
        while(current_gate.prev):
            prev_count += 1
        next_count = 0
        while(current_gate.next):
            next_count += 1
        self.gain = prev_count - next_count
    
    def set_next(self, next_gate):
        self.next = next_gate
        if next_gate:
            next_gate.prev = self

    def set_prev(self, prev_gate):
        self.prev = prev_gate
        if prev_gate:
            prev_gate.next = self

In [3]:
gate1 = gate_dlinkedList(1)
gate2 = gate_dlinkedList(2)
gate3 = gate_dlinkedList(3)
gate4 = gate_dlinkedList(4)
gate5 = gate_dlinkedList(5)
gate6 = gate_dlinkedList(6)

print(gate1.gate_id)


1


In [6]:
class FM_algo:
    def __init__(self ):
        self.circuit_data = None
        self.vertex = None
        self.best_gain = None
    def read_circuit(self, circuit_folder_path):
        net_file = None
        netD_file = None
        are_file = None
        for file in os.listdir(circuit_folder_path):
            if file.endswith('.net'):
                net_file = os.path.join(circuit_folder_path, file)
            if file.endswith('.netD'):
               netD_file = os.path.join(circuit_folder_path ,file)
            if file.endswith('.are'):
               are_file = os.path.join(circuit_folder_path, file)
                    
    ## UPDATE 2/21 01:00 JIMMY
        if net_file and netD_file and are_file:
            self.circuit_data = self.parse_net_files(net_file, netD_file, are_file)
            return self.circuit_data
        else:
            print("Error Message: Missing Files")

    def parse_net_files(self, net_file, netD_file, are_file):
        # network information
        nets, connections, nodes, modules, pad_offset = self.read_net_file(net_file)
        # module areas
        areas = self.read_are_file(are_file)
        # network information with pin directions
        netD_nets = self.read_netD_file(netD_file)
        circuit_data = {
            'nets': nets,
            'areas': areas,
            'netD_nets': netD_nets,
            'connections': int(connections),
            'nodes': int(nodes),
            'modules': int(modules),
            'pad_offset': int(pad_offset)
        }
        return circuit_data
    
    def read_net_file(self, circuit_folder_path):
        with open(circuit_folder_path, 'r') as file:
            lines = file.readlines()
        circuit_info = tuple(item.replace('\n', '') for item in lines[0:5])
        #Ex: ('0', '13', '5', '7', '3')
        _, connections, nodes, modules, pad_offset  = circuit_info

        lines = lines[5:]
        net = {}
        current_key = None
        for line in lines:
            tokens = line.split()
            if tokens[1] == 's':
                current_key = tokens[0]
                net[current_key] = []
            elif tokens[1] == 'l':
                net[current_key].append(tokens[0])
    
        return net, connections, nodes, modules, pad_offset


    def read_are_file(self, circuit_folder_path):
        with open(circuit_folder_path, 'r') as file:
            lines = file.readlines()
        module_areas = {}
        for line in lines:
            module_id, area = line.split()
            module_areas[module_id] = int(area)
        return module_areas

    def read_netD_file(self, circuit_folder_path):
        with open(circuit_folder_path, 'r') as file:
            lines = file.readlines()
        nets = []
        current_net = []
        for line in lines:
            if line.startswith('p') or line.startswith('a'):
                module_type, module_id, pin_direction = line.split()
                current_net.append((module_type, module_id, pin_direction))
            elif line.startswith('s'):
                if current_net:
                    nets.append(current_net)
                    current_net = []
        if current_net:
            nets.append(current_net)
        return nets




In [7]:
#### TEST FUNCTION ####
#### inputs from https://vlsicad.ucsd.edu/UCLAWeb/cheese/ispd98.html#Benchmark%20File%20Format ####
#### example with 4 cells, 3 pads, 5 nets and 13 pins ####
circuit_folder_path = "test_simple"
fm = FM_algo()
circuit_data = fm.read_circuit(circuit_folder_path)
print(circuit_data)


{'nets': {'p1': ['a0', 'a1'], 'a0': ['a2', 'a3'], 'a1': ['a2', 'a3'], 'a2': ['p2'], 'a3': ['p3']}, 'areas': {'a0': 1, 'a1': 3, 'a2': 4, 'a3': 2, 'p1': 0, 'p2': 0, 'p3': 0}, 'netD_nets': [[('p1', 's', 'O'), ('a0', 'l', 'I'), ('a1', 'l', 'I'), ('a0', 's', 'O'), ('a2', 'l', 'I'), ('a3', 'l', 'I'), ('a1', 's', 'O'), ('a2', 'l', 'I'), ('a3', 'l', 'I'), ('a2', 's', 'O'), ('p2', 'l', 'I'), ('a3', 's', 'O'), ('p3', 'l', 'I')]], 'connections': 13, 'nodes': 5, 'modules': 7, 'pad_offset': 3}


In [8]:
circuit_data['nets']

{'p1': ['a0', 'a1'],
 'a0': ['a2', 'a3'],
 'a1': ['a2', 'a3'],
 'a2': ['p2'],
 'a3': ['p3']}

In [9]:
circuit_data['areas']

{'a0': 1, 'a1': 3, 'a2': 4, 'a3': 2, 'p1': 0, 'p2': 0, 'p3': 0}

In [11]:
circuit_data['netD_nets']

[[('p1', 's', 'O'),
  ('a0', 'l', 'I'),
  ('a1', 'l', 'I'),
  ('a0', 's', 'O'),
  ('a2', 'l', 'I'),
  ('a3', 'l', 'I'),
  ('a1', 's', 'O'),
  ('a2', 'l', 'I'),
  ('a3', 'l', 'I'),
  ('a2', 's', 'O'),
  ('p2', 'l', 'I'),
  ('a3', 's', 'O'),
  ('p3', 'l', 'I')]]

In [ ]:
def main():
    pass

if __name__ == "__main__":
    main()

In [ ]:
print(circuit_data['areas'])
print(circuit_data['netD_nets'])
print(circuit_data['nets'])